In [ ]:
import libsbml
import importlib
import amici
import amici.plotting
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import scipy.stats

from gm import gm
from gm_Prep import gm_Prep

In [ ]:
# np.random.seed(1337)

In [ ]:
flagD = 1
th = 24
ts = 30
tsup = 0
NSteps = th*3600/ts
NSteps = int(NSteps)

# SBML model we want to import
sbml_file = 'SPARCED_Brep.xml'
# Name of the model that will also be the name of the python module
model_name = sbml_file[0:-4]
# Directory to which the generated model code is written
model_output_dir = model_name

sys.path.insert(0, os.path.abspath(model_output_dir))
model_module = importlib.import_module(model_name)
model = model_module.getModel()
solver = model.getSolver() # Create solver instance
solver.setMaxSteps = 1e10
model.setTimepoints(np.linspace(0,ts)) # np.linspace(0, 30) # set timepoints

kGsRead = pd.read_csv('OmicsData.txt',header=0,index_col=0,sep="\t")
gExp_mpc = np.float64(kGsRead.values[:,0])
mExp_mpc = np.float64(kGsRead.values[:,1])
kGin = np.float64(kGsRead.values[:,2])
kGac = np.float64(kGsRead.values[:,3])
kTCleak = np.float64(kGsRead.values[:,4])
kTCmaxs = np.float64(kGsRead.values[:,5])
kTCd = np.float64(kGsRead.values[:,6])
kmRNAs = kGsRead.values[:,7]

# Read-in the activators matrix and assign concentrations of activators
TAsRead = pd.read_csv('TAs.csv',header=0,index_col=0)
TAs0 = np.float64(TAsRead.values)
# Read-in the repressors matrix and assign concentrations of repressors
TRsRead = pd.read_csv('TRs.csv',header=0,index_col=0)
TRs0 = np.float64(TRsRead.values)

In [ ]:
genedata, GenePositionMatrix, AllGenesVec, xgac_mpc_D, xgin_mpc_D, xgac_mpc, xgin_mpc, kTCleak2 \
= gm_Prep(flagD, gExp_mpc, mExp_mpc, kGin, kGac, kTCleak, kTCmaxs, kTCd)

In [ ]:
Vn = 1.75E-12
Vc = 5.25E-12
    
params2chg = ['k313_1','k314_1','k315_1'] # DNA damage related parameters
params2chgVal = []
for pp in params2chg:
    params2chgVal.append(model.getFixedParameterById(pp))

# SSB damage simulation
startTime = datetime.now()
tout_all = np.arange(0,th*3600+1,30)
spdata0 = pd.read_csv('Species_Brep.txt',header=0,index_col=0,sep="\t")
spdata = np.double(spdata0.values[:,1])
spdata[31] = 25.0 # damageSSB

condsSSB = []
m = [0,0.15]
for ii in range(len(m)): 
    model2 = model
    for count,pp in enumerate(params2chg):
        model2.setFixedParameterById(pp,params2chgVal[count]*m[ii])
    xoutS_all = np.zeros(shape=(NSteps+1,len(spdata)))
    xoutS_all[0,:] = spdata
    for qq in range(NSteps):
        genedata,AllGenesVec = gm(flagD,ts,genedata,xoutS_all[qq,:],Vn,Vc,kGin,kGac, \
                                   kTCmaxs,kTCleak,kTCd,AllGenesVec,GenePositionMatrix,TAs0,TRs0)
        for ii,kk in enumerate(kmRNAs):
            model2.setFixedParameterById(kk,genedata[ii+282]*(1E9/(Vc*6.023E+23)))
        model2.setInitialStates(xoutS_all[qq,:])
        rdata = amici.runAmiciSimulation(model2, solver)  # Run simulation
        xoutS_all[qq+1,:] = rdata['x'][-1,:]
    condsSSB.append(xoutS_all)
    print(datetime.now() - startTime)
    
# DSB damage simulation
startTime = datetime.now()
spdata = np.double(spdata0.values[:,1])
spdata[30] = 25.0 # damageDSB
condsDSB = []
m = [0.0,5.0];
for ii in range(len(m)): 
    model2 = model
    for count,pp in enumerate(params2chg):
        model2.setFixedParameterById(pp,params2chgVal[count]*m[ii])
    
    xoutS_all = np.zeros(shape=(NSteps+1,len(spdata)))
    xoutS_all[0,:] = spdata

    for qq in range(NSteps):
        genedata,AllGenesVec = gm(flagD,ts,genedata,xoutS_all[qq,:],Vn,Vc,kGin,kGac, \
                                   kTCmaxs,kTCleak,kTCd,AllGenesVec,GenePositionMatrix,TAs0,TRs0)
        for ii,kk in enumerate(kmRNAs):
            model2.setFixedParameterById(kk,genedata[ii+282]*(1E9/(Vc*6.023E+23)))
        model2.setInitialStates(xoutS_all[qq,:])
        rdata = amici.runAmiciSimulation(model2, solver)  # Run simulation
        xoutS_all[qq+1,:] = rdata['x'][-1,:]
    condsDSB.append(xoutS_all)
    print(datetime.now() - startTime)

# SSB+DSB damage simulate
startTime = datetime.now()
spdata = np.double(spdata0.values[:,1])
spdata[[30,31]] = 25.0 # damageSSB+DSB
condsDSSB = []
m = [0.0,5.0];
for ii in range(len(m)): 
    model2 = model
    for count,pp in enumerate(params2chg):
        model2.setFixedParameterById(pp,params2chgVal[count]*m[ii])
    
    xoutS_all = np.zeros(shape=(NSteps+1,len(spdata)))
    xoutS_all[0,:] = spdata

    for qq in range(NSteps):
        genedata,AllGenesVec = gm(flagD,ts,genedata,xoutS_all[qq,:],Vn,Vc,kGin,kGac, \
                                   kTCmaxs,kTCleak,kTCd,AllGenesVec,GenePositionMatrix,TAs0,TRs0)
        for ii,kk in enumerate(kmRNAs):
            model2.setFixedParameterById(kk,genedata[ii+282]*(1E9/(Vc*6.023E+23)))
        model2.setInitialStates(xoutS_all[qq,:])
        rdata = amici.runAmiciSimulation(model2, solver)  # Run simulation
        xoutS_all[qq+1,:] = rdata['x'][-1,:]
    condsDSSB.append(xoutS_all)
    print(datetime.now() - startTime)

In [ ]:
with pd.ExcelWriter('Fig3E_sims_24hr_det.xlsx') as writer:
    condsDF1 = pd.DataFrame(data=condsSSB[0],columns=[ele for ele in model.getStateIds()]) 
    condsDF1.to_excel(writer, sheet_name='Sheet_SSB1')
    condsDF2 = pd.DataFrame(data=condsSSB[1],columns=[ele for ele in model.getStateIds()]) 
    condsDF2.to_excel(writer, sheet_name='Sheet_SSB2')
    
    condsDF3 = pd.DataFrame(data=condsDSB[0],columns=[ele for ele in model.getStateIds()]) 
    condsDF3.to_excel(writer, sheet_name='Sheet_DSB1')
    condsDF4 = pd.DataFrame(data=condsDSB[1],columns=[ele for ele in model.getStateIds()]) 
    condsDF4.to_excel(writer, sheet_name='Sheet_DSB2')
    
    condsDF5 = pd.DataFrame(data=condsDSSB[0],columns=[ele for ele in model.getStateIds()]) 
    condsDF5.to_excel(writer, sheet_name='Sheet_DSSB1')
    condsDF6 = pd.DataFrame(data=condsDSSB[1],columns=[ele for ele in model.getStateIds()]) 
    condsDF6.to_excel(writer, sheet_name='Sheet_DSSB2')

In [ ]:
plt.figure(figsize=(7, 4))
plt.plot(tout_all/3600.0,(((condsSSB[0][:,2]))),linewidth=4)
plt.plot(tout_all/3600.0,(((condsSSB[1][:,2]))),linewidth=4)
plt.xlabel('Time (hrs)')
plt.ylabel('p53 active (nM)', multialignment='center')
plt.legend(['Wihout repair','With repair'])
plt.grid(True)
plt.xlim(0, 24)
plt.ylim(0, 400)
plt.xticks(np.arange(0,24,step=6))
plt.savefig('p53act_SSB.png')

plt.figure(figsize=(7, 4))
plt.plot(tout_all/3600.0,(((condsDSB[0][:,2]))),linewidth=4)
plt.plot(tout_all/3600.0,(((condsDSB[1][:,2]))),linewidth=4)
plt.xlabel('Time (hrs)')
plt.ylabel('p53 active (nM)', multialignment='center')
plt.legend(['Wihout repair','With repair'])
plt.grid(True)
plt.xlim(0, 24)
plt.ylim(0, 1000)
plt.xticks(np.arange(0,24,step=6))
plt.savefig('p53act_DSB.png')

plt.figure(figsize=(7, 4))
plt.plot(tout_all/3600.0,(((condsDSSB[0][:,2]))),linewidth=4)
plt.plot(tout_all/3600.0,(((condsDSSB[1][:,2]))),linewidth=4)
plt.xlabel('Time (hrs)')
plt.ylabel('p53 active (nM)', multialignment='center')
plt.legend(['Wihout repair','With repair'])
plt.grid(True)
plt.xlim(0, 24)
plt.ylim(0, 1200)
plt.xticks(np.arange(0,24,step=6))
plt.savefig('p53act_both.png')